<a href="https://colab.research.google.com/github/swansonk14/chemprop-intro/blob/master/lab1/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Random Forest on Morgan Fingerprint

In [2]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c rdkit rdkit scikit-learn

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

--2018-12-27 06:04:04--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.17.110.77, 104.17.111.77, 104.17.109.77, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.17.110.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62574861 (60M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  59.68M   141MB/s    in 0.4s    

2018-12-27 06:04:05 (141 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [62574861/62574861]

PREFIX=/usr/local
reinstalling: python-3.7.0-hc3d631a_0 ...
Python 3.7.0
reinstalling: ca-certificates-2018.03.07-0 ...
reinstalling: conda-env-2.6.0-1 ...
reinstalling: libgcc-ng-8.2.0-hdf63c60_1 ...
reinstalling: libstdcxx-ng-8.2.0-hdf63c60_1 ...
reinstalling: libffi-3.2.1-hd88cf55_4 ...
reinstalling: ncurses-6.1-hf484d3e_0 ...
reinstalling: openssl-1.0.2p-h14c3975_0 ...
reinstalling: xz-5.2.4-h14c397

In [0]:
import math
import os
import random
from typing import Union, List, Dict

import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def get_data(data_dir: str, dataset: str = 'delaney', split: str = None) -> List[Dict[str, Union[str, float]]]:
  data_name = '{}_{}.csv'.format(dataset, split) if split is not None else '{}.csv'.format(dataset)
  data_path = os.path.join(data_dir, data_name)
  with open(data_path) as f:
    f.readline()
    data = []
    for line in f:
      smiles, target = line.strip().split(',')
      target = float(target)
      data.append({
          'smiles': smiles,
          'target': target
      })
      
  return data

In [0]:
data_dir = '/content/gdrive/My Drive/Research/Chemistry/Classes'
train, test = get_data(data_dir, split='train'), get_data(data_dir, split='test')

In [0]:
def morgan_fingerprint(smiles: str, radius: int = 3, num_bits: int = 2048) -> np.ndarray:
  mol = Chem.MolFromSmiles(smiles)
  morgan_vect = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=num_bits)
  morgan_fp = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(morgan_vect, morgan_fp)
  
  return morgan_fp

In [0]:
for dataset in [train, test]:
  for datapoint in dataset:
    datapoint['morgan'] = morgan_fingerprint(datapoint['smiles'])

In [0]:
X_train, Y_train = [datapoint['morgan'] for datapoint in train], [datapoint['target'] for datapoint in train]
X_test, Y_test = [datapoint['morgan'] for datapoint in test], [datapoint['target'] for datapoint in test]

In [66]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [70]:
def rmse(targets: List[float], preds: List[float]) -> float:
    return math.sqrt(mean_squared_error(targets, preds))

Y_pred = model.predict(X_test)
print('rmse = {:.4f}'.format(rmse(Y_test, Y_pred)))

rmse = 1.2125
